In [8]:
import tkinter as tk
import numpy as np
import pandas as pd
import tkinter.ttk as ttk

In [9]:
# debug
import import_ipynb
from poem_database import PoemDatabase
poem_db = PoemDatabase()

C:\Users\45887\AppData\Local\conda\conda\envs\torch\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)
<string>:96: RuntimeWarning: divide by zero encountered in reciprocal


In [10]:
class Subscription:
    def __init__(self, poem_database, author="", keyword=[]):
        self.author = author
        self.keyword = keyword
        self.content, self.simword = poem_database.query_poem(author, keyword)

In [12]:
class MainPanel:
    def __init__(self, poem_db):
        self.poem_db = poem_db
        self.subscriptions = []
    
    def start(self):
        """
        显示主界面
        """
        self.root = tk.Tk()
        self.root.geometry('600x600')
        self.root.title("古诗订阅")
        
        self.label = tk.Label(self.root, text="我的订阅", font=(None,12)).pack(pady=20)
        for s, idx in enumerate(self.subscriptions):
            title = " ".join([s.author] + s.keyword)
            title = "订阅" + str(idx) + ": " + title
            display_button = tk.Button(self.root, text=s.title+"", font=(None,12), command=self.display_subscription).pack(pady=20)
        self.add_button = tk.Button(self.root, text="新增订阅", font=(None,12), command=self.add_subscription).pack(pady=40)
        
        self.root.mainloop()
        
    
    def add_subscription(self):
        """
        用户点击"新增订阅"按钮，显示该界面
        """
        self.add_panel = tk.Tk()
        self.add_panel.geometry('600x600')
        self.add_panel.title("新增订阅")
        
        label = tk.Label(self.add_panel, text="新增订阅:", font=(None, 10)).pack(pady=20)
        
        author_frame = tk.Frame(self.add_panel)
        label_author = tk.Label(author_frame, text="作者：", font=(None, 10)).pack(side="left", pady=20)
        self.author_entry = tk.Entry(author_frame)
        self.author_entry.pack(side="right", pady=20)
        author_frame.pack(pady=10)

        self.keyword_entry = []
        for i in range(3):
            keyword_frame = tk.Frame(self.add_panel)
            label_keyword = tk.Label(keyword_frame, text="关键词：", font=(None, 10)).pack(side="left", pady=20)
            self.keyword_entry.append(tk.Entry(keyword_frame))
            self.keyword_entry[-1].pack(side="right",pady=20)
            keyword_frame.pack(pady=10)
        
        confirm_button = tk.Button(self.add_panel, text="确认", font=(None, 10), command=self.confirm_subscription)
        confirm_button.pack(pady=40)
        
        self.hint_label = tk.Label(self.add_panel, text="", font=(None, 10))
        self.hint_label.pack()
        
        
    def confirm_subscription(self):
        """
        将订阅添加到主界面中
        """
        author = self.author_entry.get()
        keyword = [entry.get() for entry in self.keyword_entry]
        keyword = [x for x in keyword if x != ""]
        
        if author == "" and keyword == []:
            tmp_tk = tk.Tk()
            tmp_tk.geometry("300x200")
            tk.Label(tmp_tk, text="害没查呢！", font=(None, 20)).pack(pady=20)
            tk.Button(tmp_tk, text="确认", font=(None, 12), command=tmp_tk.destroy).pack(pady=20) 
            return           
        
        self.subscriptions.append(Subscription(self.poem_db, author, keyword))
        
        tmp_tk = tk.Tk()
        tmp_tk.geometry("300x200")
        tk.Label(tmp_tk, text="订阅成功", font=(None, 20)).pack(pady=20)
        tk.Button(tmp_tk, text="确认", font=(None, 12), command=tmp_tk.destroy).pack(pady=20)        
        
        self.add_panel.destroy()
        self.update_main_panel()
        
    
    def update_main_panel(self):
        """
        用户新增订阅后，更新主界面
        """
        for widget in self.root.winfo_children():
            widget.destroy()
        self.label = tk.Label(self.root, text="我的订阅", font=(None,12)).pack(pady=20)
        for idx, s in enumerate(self.subscriptions):
            title = " ".join([s.author] + s.keyword)
            title = "订阅" + str(idx+1) + ": " + title
            display_button = tk.Button(self.root, text=title, font=(None,12), command=lambda s=s: self.display_subscription(s)).pack(pady=20)
        self.add_button = tk.Button(self.root, text="新增订阅", font=(None,12), command=self.add_subscription).pack(pady=40)
        self.root.update()
        
    
    def display_subscription(self, subscription):
        """
        用notebook展示当前的订阅，推荐使用键盘来浏览
        """
        
        display_panel = tk.Tk()
        display_panel.geometry("600x600")
        display_panel.title("查看订阅——可使用(Shift-)Control-Tab前后翻页浏览内容")
        
        notebook = ttk.Notebook(display_panel)
        
        content = subscription.content.content
        cur_content = []
        page_size = 5
        cnt = 0
        
        for i in range(0, len(content), page_size):
            if i + page_size > len(content):
                cur_content = content[i:]
            else:
                cur_content = content[i: i+page_size]
            cnt += 1
            tab = tk.Frame(notebook)
            notebook.add(tab, text=f"[{cnt}]")
            for poem in cur_content:
                # 配置text的内容
                text = tk.Text(tab, width=30, height=6)
                text.insert("insert","\n".join(poem))
                
                # 对text做关键词高亮
                for word in subscription.simword:
                    if word in subscription.keyword:
                        search_tag = "search_key"
                        bg = "yellow"
                    else:
                        search_tag = "search_sim"
                        bg = "cyan"
                    
                    start = 1.0
                    while True:
                        pos = text.search(word, start, stopindex="end")
                        if not pos:
                            break
                        text.tag_add(search_tag, pos, f"{pos} + {len(word)}c")
                        text.tag_config(search_tag, background=bg)
                        start = pos + "+1c"  # 将 start 指向下一个字符
                
                text["state"] = "disabled"
                text.pack()
            
                
        
        notebook.enable_traversal()
        notebook.pack()
        


In [13]:
# main_panel = MainPanel(poem_db)
# main_panel.start()